In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [11]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [12]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [13]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv0 = nn.Sequential(         
            nn.Conv2d(32, 32*7, 1, 1, padding = "same",),
            nn.BatchNorm2d(32*7),
            nn.ReLU(),
            nn.Conv2d(32*7, 32*7, 3, 1, padding = "same",),
            nn.BatchNorm2d(32*7),
            nn.ReLU(),
            nn.Conv2d(32*7, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2) 
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        identity = x
        x = self.conv0(x)
        x = identity + x
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [14]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

CNN_model(
  (conv0): Sequential(
    (0): Conv2d(32, 224, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(224, 224, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(224, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(


In [16]:
from torchsummary import summary
summary(model,(3,56,56))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 56, 56]             896
              ReLU-2           [-1, 32, 56, 56]               0
         MaxPool2d-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 64, 28, 28]          18,496
              ReLU-5           [-1, 64, 28, 28]               0
         MaxPool2d-6           [-1, 64, 14, 14]               0
            Conv2d-7           [-1, 32, 14, 14]          18,464
              ReLU-8           [-1, 32, 14, 14]               0
         MaxPool2d-9             [-1, 32, 7, 7]               0
           Conv2d-10            [-1, 224, 7, 7]           7,392
      BatchNorm2d-11            [-1, 224, 7, 7]             448
             ReLU-12            [-1, 224, 7, 7]               0
           Conv2d-13            [-1, 224, 7, 7]         451,808
      BatchNorm2d-14            [-1, 22

# Step 3 Pass in the model to start training and testing

In [6]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/30]	|	Step: [9/27]	|	Loss: 2.9629
Epoch: [1/30]	|	Step: [18/27]	|	Loss: 2.9993
Epoch: [1/30]	|	Step: [27/27]	|	Loss: 2.6470
10.820451736450195
epoch: 01 | Accuracy: 10.82 Loss: 80.1833

Epoch: [2/30]	|	Step: [9/27]	|	Loss: 2.7274
Epoch: [2/30]	|	Step: [18/27]	|	Loss: 2.4395
Epoch: [2/30]	|	Step: [27/27]	|	Loss: 2.0834
26.753864288330078
epoch: 02 | Accuracy: 26.75 Loss: 67.2274

Epoch: [3/30]	|	Step: [9/27]	|	Loss: 2.0099
Epoch: [3/30]	|	Step: [18/27]	|	Loss: 1.6978
Epoch: [3/30]	|	Step: [27/27]	|	Loss: 1.6421
49.10820388793945
epoch: 03 | Accuracy: 49.11 Loss: 50.0370

Epoch: [4/30]	|	Step: [9/27]	|	Loss: 1.2944
Epoch: [4/30]	|	Step: [18/27]	|	Loss: 1.1414
Epoch: [4/30]	|	Step: [27/27]	|	Loss: 1.2281
66.23067474365234
epoch: 04 | Accuracy: 66.23 Loss: 32.8707

Epoch: [5/30]	|	Step: [9/27]	|	Loss: 0.6941
Epoch: [5/30]	|	Step: [18/27]	|	Loss: 0.7568
Epoch: [5/30]	|	Step: [27/27]	|	Loss: 0.3794
81.33174896240234
epoch: 05 | Accuracy: 81.33 Loss: 19.6119

Epoch: [6/30]	|	Step: [

In [7]:
print(Accy_list)

[10.820451736450195, 26.753864288330078, 49.10820388793945, 66.23067474365234, 81.33174896240234, 90.96314239501953, 93.935791015625, 98.33531188964844, 99.28656005859375, 99.64328002929688, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [8]:
print(Loss_list)

[80.18330574035645, 67.22742295265198, 50.03700542449951, 32.87070423364639, 19.61191913485527, 11.794018357992172, 7.696299538016319, 3.7660284154117107, 2.0435632467269897, 1.1160530485212803, 0.5020411210134625, 0.3753550285473466, 0.28561934223398566, 0.21492305770516396, 0.14627696410752833, 0.12427938380278647, 0.10732339322566986, 0.09825063589960337, 0.0864284138660878, 0.08338139089755714, 0.07095375179778785, 0.06482598779257387, 0.0579795598750934, 0.055343180312775075, 0.04821658931905404, 0.043491599382832646, 0.04395600192947313, 0.038228072226047516, 0.03468917869031429, 0.03316859435290098]


In [9]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 94.29 %, Loss: 6.6065 
